In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score 
# import seaborn as sns

In [86]:
class RidgeRegression:
  def __init__(self, data):
    self.data = data
    self.alpha = 0.1 
    self.w = None
  
  def fit(self, X, y):
    X = np.c_[np.ones(X.shape[0]), X]
    xTranspose = X.T
    A = xTranspose @ X
    I = np.identity(A.shape[0])
    c = xTranspose @ y
    self.w = np.linalg.inv(A + self.alpha * I) @ c
  
  def predict(self, X):
    X = np.c_[np.ones(X.shape[0]), X]
    return X @ self.w
    

In [ ]:
class LassoRegression:
  def __init__(self, data):
    self.data = data
    self.alpha = 0.1
    self.w = None
  
  def fit(self, X, y):
    # Placeholder for Lasso regression fitting logic
    pass
  
  def predict(self, X):
    # Placeholder for Lasso regression prediction logic
    pass
    

In [87]:
data = pd.read_csv("../data/diabetes.csv")
# sns.heatmap(data.corr(), annot=True)
target = data["target"]
data.drop(columns=["target"], inplace=True)
model = RidgeRegression(data)
model.fit(data, target)
y_pred = model.predict(data)
mse = mean_squared_error(target, y_pred)
r2 = r2_score(target, y_pred)
print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 2890.4524762078877
R2 Score: 0.5125617905814859
